In [3]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from md04_func import *

df=pd.DataFrame(md04['data'], columns=md04['header'])
df['cum']=df.Change.cumsum()
df.cum.plot(x="Date")
plt.plot(df.Date, df.cum)
target=df.mask('Type','ShipNt')
plt.plot(target.Date, target.cum)

filelist=[]
f=open("MD04list.txt","r", encoding="UTF-8")
list=f.readlines()
for i in list :        
    document=fileRead(i.rstrip("\n"))
    md04=parsingDoc(document)
    PD=pd.DataFrame(md04["data"], columns=md04["header"]) 
    
    
    
fig = plt.figure()
ax= PD['Rec/reqd Qty'].plot(x="Date", kind="bar", title=md04['meta'][0])
ax2 = ax.twinx()
ax2.plot(ax.get_xticks(),PD['AvailQty'])

In [4]:
category = {
    'ShipNt':"Receipt",
    'SchLne':"Receipt",
    'Order' : 'Reqmt',
    'Stock' : 'Receipt',
    'CStock' : 'Reqmt',
    'Ordres' : 'Reqmt',
    'SafeSt' : '',
    'StLcSt' : '',
    'PurRqs' : '',
    'Deliv.' : 'Reqmt'
}
header=["Partnr","SO_Date","SO_MRP", "SO_NR", "SO_POS","SO_QTY", "PO_DATE","PO_MRP","PO_NR", "PO_POS","PO_QTY" ]
total=[]


## TEST

In [5]:
def kmatParser(df) : 
    MappedResult=[]
    idx=mask(df, "Type", "CStock").index
    CStockReqmt=df.iloc[idx]
    CStockReceipt=df.iloc[idx+1]

    if len(CStockReqmt) != len(CStockReceipt) : 
        print("Please check CStock as rows are not matched")
    
    for i,val in enumerate(CStockReceipt.iterrows()):
        MappedResult.append (CStockReqmt.iloc[i].tolist()[0:6] + (CStockReceipt.iloc[i].tolist()[1:6]))
    
    return MappedResult, df.drop(index=idx+1).drop(index=idx)   

In [6]:
def fehaParser(df) :
    ReceiptQueue=[]
    ReqmtQueue=[]
    MappedResult=[]
    flag=0

    
    #KMAT PARSER
    MappedResult, df = kmatParser(df)    
    for index, line in df.iterrows() :    
        if line[5]==0  and line[2]== 'Stock' : continue
        if category[line[2]] == 'Receipt' :        
            ReceiptQueue.append(line)
        if category[line[2]] == 'Reqmt' :
            ReqmtQueue.append(line)
        else :
            continue
            
    #If df only has kmat (CStock, then it return )
    if len(df)<=1 :        
        return MappedResult    
    else : 
        Receipt = ReceiptQueue.pop(0)
        Reqmt = ReqmtQueue.pop(0)  

        while True : 
            try :        
                print (Receipt[5], Reqmt[5])
                if Receipt[5] + Reqmt[5] == 0 :                    
                    flag=0
                    MappedResult.append([Reqmt[0], Reqmt[1], Reqmt[2], Reqmt[3], Reqmt[4], Reqmt[5],  Receipt[1], Receipt[2], Receipt[3], Receipt[4], Receipt[5]])    
                    Receipt=[]            
                    Receipt = ReceiptQueue.pop(0)
                    Reqmt=[]
                    Reqmt = ReqmtQueue.pop(0)  

                if Receipt[5] + Reqmt[5] > 0 :                           
                    flag=1
                    MappedResult.append([Reqmt[0], Reqmt[1], Reqmt[2], Reqmt[3], Reqmt[4], Reqmt[5],  Receipt[1], Receipt[2], Receipt[3], Receipt[4], -Reqmt[5]])    
                    Receipt[5] +=Reqmt[5]            
                    Reqmt = ReqmtQueue.pop(0)  


                if Receipt[5] + Reqmt[5] < 0 :                                
                    flag=-1
                    MappedResult.append([Reqmt[0], Reqmt[1], Reqmt[2], Reqmt[3], Reqmt[4], -Receipt[5],  Receipt[1], Receipt[2], Receipt[3], Receipt[4], Receipt[5]])
                    Reqmt[5] +=Receipt[5]
                    Receipt = ReceiptQueue.pop(0)

            #if no elment for popping.  
            except Exception as e:   

                if flag == -1:
                    MappedResult.append([Reqmt[0], Reqmt[1], Reqmt[2], Reqmt[3], Reqmt[4], Reqmt[5],'','' ,'' ,'' ,'' ]) 
                elif flag == 1:
                    MappedResult.append([Receipt[0], '','','','','',  Receipt[1], Receipt[2], Receipt[3], Receipt[4], Receipt[5]])          

                elif flag == 0:
                    print("flagflag")
                    if len(Reqmt) ==0 : 
                        #print("Reqmt empty")
                        MappedResult.append([Receipt[0], '','','','','',  Receipt[1], Receipt[2], Receipt[3], Receipt[4], Receipt[5]])          
                    elif len(Receipt) ==0 :                
                        #print("Receipt empty")
                        MappedResult.append([Reqmt[0], Reqmt[1], Reqmt[2], Reqmt[3], Reqmt[4], Reqmt[5],'','' ,'' ,'' ,'' ]) 

                #print("ReqmtQueue :", len(ReqmtQueue), "ReceiptQueue :", len(ReceiptQueue))
                for Reqmt in ReqmtQueue :
                    MappedResult.append([Reqmt[0], Reqmt[1], Reqmt[2], Reqmt[3], Reqmt[4], Reqmt[5],'','' ,'' ,'' ,'' ]) 

                for Receipt in ReceiptQueue :
                    MappedResult.append([Reqmt[0], '','','','','',  Receipt[1], Receipt[2], Receipt[3], Receipt[4], Receipt[5]])          
                print ("!!!!!!!!!!!!!!!!!!!!!!!!")
                print (MappedResult)
                print ("!!!!!!!!!!!!!!!!!!!!!!!!")
                return MappedResult

## LIST 

In [7]:
total=[]
filelist=[]
f=open("MD04list.txt","r", encoding="UTF-8")
list=f.readlines()
for i in list :      
    md04=parsingDoc(fileRead(i.replace("\n","")))    
    PD=pd.DataFrame(md04["data"], columns=md04["header"])   
    #PD.plot(x="Date", kind="bar", title=md04['meta'][0])
    df=PD.iloc[:,0:6]    
    total.append(pd.DataFrame(fehaParser(df), columns=header))

['543906', '2018/05/11', 'Stock', 0, 0, 0, 0]
['543906', '2018/05/11', 'CStock', '5236281646', '000010', 0, 0]
['543906', '2018/05/15', 'ShipNt', '4601956922', '00001', 2, 0]
['543906', '2018/05/11', 'CStock', '5236285365', '000010', 0, 0]
['543906', '2018/05/18', 'ShipNt', '4601971081', '00001', 1, 0]
['543906', '2018/05/11', 'CStock', '5236289731', '000020', 0, 0]
['543906', '2018/05/24', 'SchLne', '4601988901', '00001', 1, 0]
['543906', '2018/05/11', 'CStock', '5236289741', '000020', 0, 0]
['543906', '2018/08/10', 'SchLne', '4601989219', '00001', 1, 0]
['543908', '2018/05/11', 'Stock', 0, 0, 0, 0]
['543908', '2018/05/11', 'CStock', '5236282681', '000360', 0, 0]
['543908', '2018/07/18', 'ShipNt', '4601960196', '00004', 1, 0]
['543908', '2018/05/11', 'CStock', '5236282681', '000820', 0, 0]
['543908', '2018/06/08', 'ShipNt', '4601960196', '00006', 1, 0]
['543908', '2018/05/11', 'CStock', '5236285365', '000020', 0, 0]
['543908', '2018/05/18', 'ShipNt', '4601971081', '00002', 1, 0]
['543

In [8]:
len(total)

8

In [9]:
total[0]

,Partnr,SO_Date,SO_MRP,SO_NR,SO_POS,SO_QTY,PO_DATE,PO_MRP,PO_NR,PO_POS,PO_QTY
0,543906,2018/05/11,CStock,5236281646,000010,0,2018/05/15,ShipNt,4601956922,00001,2
1,543906,2018/05/11,CStock,5236285365,000010,0,2018/05/18,ShipNt,4601971081,00001,1
2,543906,2018/05/11,CStock,5236289731,000020,0,2018/05/24,SchLne,4601988901,00001,1
3,543906,2018/05/11,CStock,5236289741,000020,0,2018/08/10,SchLne,4601989219,00001,1


In [10]:
total[1]

,Partnr,SO_Date,SO_MRP,SO_NR,SO_POS,SO_QTY,PO_DATE,PO_MRP,PO_NR,PO_POS,PO_QTY
0,543908,2018/05/11,CStock,5236282681,000360,0,2018/07/18,ShipNt,4601960196,00004,1
1,543908,2018/05/11,CStock,5236282681,000820,0,2018/06/08,ShipNt,4601960196,00006,1
2,543908,2018/05/11,CStock,5236285365,000020,0,2018/05/18,ShipNt,4601971081,00002,1
3,543908,2018/05/11,CStock,5236285365,000030,0,2018/08/07,ShipNt,4601971081,00003,1
4,543908,2018/05/11,CStock,5236289731,000040,0,2018/05/24,SchLne,4601988901,00002,1
5,543908,2018/05/11,CStock,5236289741,000040,0,2018/08/10,SchLne,4601989219,00002,1


In [11]:
total[2]

,Partnr,SO_Date,SO_MRP,SO_NR,SO_POS,SO_QTY,PO_DATE,PO_MRP,PO_NR,PO_POS,PO_QTY
0,543966,2018/05/11,Order,5236282752,000070,-1,2018/05/17,SchLne,4015506339,00012,1
1,543966,2018/05/24,Order,5236289737,000040,-1,2018/05/17,SchLne,4015506339,00012,1
2,543966,2018/05/24,Order,5236289739,000020,-1,2018/05/25,ShipNt,4045481473,00001,1
3,543966,2018/05/24,Order,5236289746,000020,-1,2018/05/25,ShipNt,4045487818,00001,1
4,543966,,,,,,2018/05/25,ShipNt,4045487818,00001,1
5,543966,,,,,,2018/05/25,ShipNt,4045492873,00001,1
6,543966,,,,,,2018/05/25,ShipNt,4045495588,00001,2
7,543966,,,,,,2018/05/25,ShipNt,4045500367,00001,3


In [12]:
total[3]

,Partnr,SO_Date,SO_MRP,SO_NR,SO_POS,SO_QTY,PO_DATE,PO_MRP,PO_NR,PO_POS,PO_QTY
0,544027,2018/05/11,Order,5236282714,000600,-1,2018/05/17,SchLne,4015506339,00031,1
1,544027,2018/05/11,Order,5236282714,000600,-1,2018/06/08,ShipNt,4045493436,00001,1
2,544027,2018/05/11,Order,5236282753,000200,-2,2018/06/08,ShipNt,4045493436,00001,2
3,544027,2018/05/11,Order,5236282753,000390,-2,2018/07/05,ShipNt,4045493418,00001,2
4,544027,2018/05/11,Order,5236282758,000790,-1,2018/07/05,ShipNt,4045493418,00001,1
5,544027,,,,,,2018/07/05,ShipNt,4045493418,00001,6


In [13]:
total[4]

,Partnr,SO_Date,SO_MRP,SO_NR,SO_POS,SO_QTY,PO_DATE,PO_MRP,PO_NR,PO_POS,PO_QTY
0,544037,2018/05/11,Order,5236282681,000520,-1,2018/07/06,ShipNt,4045494192,00001,1
1,544037,2018/05/11,Order,5236282681,000520,-1,,,,,


In [14]:
total[5]

,Partnr,SO_Date,SO_MRP,SO_NR,SO_POS,SO_QTY,PO_DATE,PO_MRP,PO_NR,PO_POS,PO_QTY
0,544038,2018/05/11,CStock,5236285597,000010,0,2018/06/04,ShipNt,4045510408,00001,1
1,544038,2018/05/11,Order,5236282752,000160,-1,2018/05/17,SchLne,4015506339,00033,1
2,544038,2018/05/24,Order,5236289739,000010,-1,2018/05/17,SchLne,4015506339,00033,1
3,544038,,,,,,2018/06/01,ShipNt,4045503748,00001,1


In [15]:
total[6]

,Partnr,SO_Date,SO_MRP,SO_NR,SO_POS,SO_QTY,PO_DATE,PO_MRP,PO_NR,PO_POS,PO_QTY
0,544049,2018/05/11,Order,5236285404,000031,-2,2018/05/17,SchLne,4015506339,00034,2
1,544049,,,,,,2018/05/28,ShipNt,4045503749,00001,2


In [16]:
total[7]

,Partnr,SO_Date,SO_MRP,SO_NR,SO_POS,SO_QTY,PO_DATE,PO_MRP,PO_NR,PO_POS,PO_QTY
0,544053,2018/05/11,Order,5236282758,000890,-1,2018/07/05,ShipNt,4045497001,00001,1
1,544053,2018/05/11,Order,5236282758,001050,-1,2018/07/05,ShipNt,4045497001,00001,1
2,544053,2018/05/11,Order,5236282758,001280,-1,2018/07/05,ShipNt,4045497001,00001,1
3,544053,,,,,,2018/07/05,ShipNt,4045497002,00001,2


In [18]:
final=pd.DataFrame()
for i in total : 
        final=final.append(i)

In [19]:
#To save excel file 
writer=pd.ExcelWriter('output.xlsx')
final.to_excel(writer,'Sheet1')
writer.save()